# Text To Class

Let's turn Natural Language into Classes by UCF_101.

First let's define some parameters of the network.

In [1]:
import torch.nn as nn

rnnType     = nn.LSTM
rnnSize     = 512
embedSize   = 256
itemLength  = 10
loadEpoch   = 20

## Dataset and Vocabulary

Since the model is trained on words, is necessary to have a vocabulary. This can be taken from the TextLoader Dataset class that have been created.

### Paths
Let's define some paths in the following cell to get the dataset file.

In [2]:
import os
from glob import glob

current_path = !pwd
current_path = current_path[0]

dataset_path = os.path.join(current_path, 'caffe', 'examples', 's2vt', 'results', '[!val]*')
dataset_path = glob(dataset_path)[0]

dataset_path

'/home/carlo/Documents/Cognitive Computing/Text2VideoGAN/caffe/examples/s2vt/results/dataset_Action_Description.txt'

### Dataset
Let's create the `TextLoader` object.

In [4]:
from TextToClass.dataloading import TextLoader

print(TextLoader.__doc__)

dataset = TextLoader(dataset_path, item_length = itemLength)


    TextLoader class that expects a file formatted in the following way:
    <<(Action||Class)Name>><<tab>><<Text>>
    E.g. Biking	A man is riding a bicycle.

    With this class you must use 0 as your Padding Character.

    Constructor:
    -----------
        path: string
            The path to the file that is formatted as written above.

        dict_file: string
            The path to the file that contains the dict_to_classes mapping.
            The mapping must be as the following line:
            E.g. 1 ApplyEyeMakeup

        item_length: int
            The requested length for each sample.

    Properties:
    ----------
        numClasses: int
            The len operator applied to the actions attribute of this class.
            
    


### Create Model

Finally let's create the model giving it the vocabulary wrapped from the dataset and let's load the state of the trained model.

In [5]:
from TextToClass.models import LSTM

network = LSTM(rnnType, rnnSize, embedSize, dataset.vocabulary )

network.loadState(loadEpoch)

## Use the Model

At the end, let's see how to use the model.

1. Input some text.
2. Use the methods as shown below.
3. Take the results.

In [9]:
import torch

humanDescription     = input('Put your input here: > ')

try:
    toForwardDescription = dataset.prepareTxtForTensor(humanDescription)
    results = network(torch.tensor(toForwardDescription).unsqueeze_(0))
    _, actionIDx = results.max(1)
    print(f'Predicted class is {dataset.getClassNameFromIndex(actionIDx.item())}')    
    
except KeyError as err:
    print('Sorry, that word is not in the vocabulary. Please try again.')

Put your input here: > a man is running
Predicted class is biking
